In [1]:
import boto3
import awswrangler as wr
import pandas as pd
import time

In [2]:
session = boto3.session.Session(profile_name='saml')
s3 = session.resource('s3')

In [3]:
parquet_bucket = 's3://xwifi-od-s3-parquet/ns/type=averagethroughput'
gis_bucket = 's3://xwifi-od-s3-transform/converted-data/gis/final'

In [4]:
result = wr.s3.list_objects(parquet_bucket, boto3_session=session)
gis_file= wr.s3.list_objects(gis_bucket, boto3_session=session)

In [5]:
result = result[0:4]
gis_file = gis_file[0]

In [7]:
gis_file

's3://xwifi-od-s3-transform/converted-data/gis/final/gis-master_snmpMAC_updated.csv'

In [12]:
gis_df = wr.s3.read_csv(gis_file, boto3_session=session)

In [13]:
gis_df = gis_df.rename(columns={"apMAC":"ap_mac_address"})

In [14]:
gis_df.head()


,ap_mac_address,Model,Street,Street2,City,State,Zipcode,Latitude,Longitude,Region,Zone,zoneType,SiteID,type,geometry,snmpReportedMac,apTZ
0,3c:7a:8a:84:1e:fc,NaN,1305 KATER ST,NaN,PHILADELPHIA,PA,19147,39.943095,-75.163699,Freedom,South St Zone 1,Secondary,NaN,home_hotspot,POINT (-75.16369899999999 39.943095),3c:7a:8a:84:1e:fd,America/New_York
1,5c:8f:e0:b1:18:02,NaN,1110 N HOPE ST,NaN,PHILADELPHIA,PA,19123,39.967898,-75.137222,Freedom,Fishtown,Secondary,NaN,home_hotspot,POINT (-75.13722199999999 39.967898),5c:8f:e0:b1:18:03,America/New_York
2,00:1d:d5:bc:e9:74,NaN,631 S 9TH ST,NaN,PHILADELPHIA,PA,19147,39.941791,-75.157001,Freedom,South St Zone 1,Secondary,NaN,home_hotspot,POINT (-75.15700099999999 39.941791),00:1d:d5:bc:e9:75,America/New_York
3,9c:34:26:71:66:d8,NaN,833 BAINBRIDGE ST,NaN,PHILADELPHIA,PA,19147,39.941846,-75.156767,Freedom,South St Zone 1,Secondary,NaN,home_hotspot,POINT (-75.156767 39.941846),9c:34:26:71:66:d9,America/New_York
4,78:23:ae:f2:47:77,NaN,1137 PINE ST,APT 103,PHILADELPHIA,PA,19107,39.945278,-75.160919,Freedom,South St Zone 1,Secondary,NaN,home_hotspot,POINT (-75.16091900000001 39.945278),78:23:ae:f2:47:78,America/New_York


In [16]:
df = wr.s3.read_parquet(result[0], boto3_session=session)

In [22]:
df.head()

,time,ap_zone,ap_ip_address,ap_mac_address,ap_model,vlan,vlan_id,client_mac_address,ingress_total_tonnage,ingress_active_time,egress_total_tonnage,egress_active_time,dt,total_tonnage,total_time
0,2020-04-30,ChestnutHill,68.80.248.134,f8:8b:37:b1:91:a7,TG1682G,XFINITY_5GHz_HHS_BWG,105,9c:04:eb:65:83:b2,19030,5272,5811,3799,05-2020,0.024841,9.071
1,2020-04-30,ChestnutHill,68.80.248.166,3c:04:61:f4:9a:3f,TG1682G,xfinitywifi_2_4GHz_HHS_BWG,102,8c:f5:a3:20:92:71,264395,15679,92616,79066,05-2020,0.357011,94.745
6,2020-04-30,ChestnutHill,68.80.248.80,80:b2:34:1c:02:34,DPC3941B,XFINITY_5GHz_HHS_BWG,105,b0:35:b5:bf:64:1c,4164389,88083,609375,88515,05-2020,4.773764,176.598
7,2020-04-30,ChestnutHill,68.80.248.89,3c:04:61:7e:ee:15,TG1682G,XFINITY_5GHz_HHS_BWG,105,00:b3:62:3e:9f:1f,1372235,62856,123193,20425,05-2020,1.495428,83.281
9,2020-04-30,ChestnutHill,68.80.248.9,a8:9f:ec:da:b9:c6,OG1600A,XFINITY_5GHz_OG,611,b0:35:b5:bf:64:1c,3812393,41365,243254,54597,05-2020,4.055647,95.962


In [19]:
df = df[df['ingress_active_time'] != 0]
df = df[df['egress_active_time'] != 0]
df['total_tonnage'] = (df['ingress_total_tonnage'] + df['egress_total_tonnage'])*1e-6
df['total_time'] = (df['ingress_active_time'] + df['egress_active_time'])*1e-3
df = df[df['total_tonnage'] >= 1e-3]

In [21]:
df = df.drop(columns=['egress_average_throughput', 'ingress_average_throughput'], axis=1)

In [27]:
def s3_read_parquet(item):
    df = wr.s3.read_parquet(item, boto3_session=session)
    df = df[df['ingress_active_time'] != 0]
    df = df[df['egress_active_time'] != 0]
    df['total_tonnage'] = (df['ingress_total_tonnage'] + df['egress_total_tonnage'])*1e-6
    df['total_time'] = (df['ingress_active_time'] + df['egress_active_time'])*1e-3
    df = df[df['total_tonnage'] >= 1e-3]
    df = df.drop(columns=['egress_average_throughput', 'ingress_average_throughput'], axis=1)
    df['average_throughput'] = (df['total_tonnage']/df['total_time'])* 1e3
    return df


In [24]:
from pathlib import Path
pickle_path = Path(Path().resolve().parents[0], 'data')

In [25]:
pickle_path

WindowsPath('C:/Users/atrakru/Documents/trajectory-stats/data')

In [46]:
df = pd.merge(s3_read_parquet(result[3]), gis_df, on='ap_mac_address')

In [42]:
df.head()

,time,ap_zone,ap_ip_address,ap_mac_address,ap_model,vlan,vlan_id,client_mac_address,ingress_total_tonnage,ingress_active_time,...,Latitude,Longitude,Region,Zone,zoneType,SiteID,type,geometry,snmpReportedMac,apTZ
0,2020-05-03 00:00:00,ChestnutHill,68.80.248.134,f8:8b:37:b1:91:a7,TG1682G,xfinitywifi_5GHz_HHS_BWG,103,9c:04:eb:65:83:b2,15071641,126292,...,40.074132,-75.200463,Freedom,Chestnut Hill,Secondary,NaN,home_hotspot,POINT (-75.200463 40.074132),f8:8b:37:b1:91:a8,America/New_York
1,2020-05-03 00:05:00,ChestnutHill,68.80.248.134,f8:8b:37:b1:91:a7,TG1682G,xfinitywifi_5GHz_HHS_BWG,103,9c:04:eb:65:83:b2,4907787,49585,...,40.074132,-75.200463,Freedom,Chestnut Hill,Secondary,NaN,home_hotspot,POINT (-75.200463 40.074132),f8:8b:37:b1:91:a8,America/New_York
2,2020-05-03 00:10:00,ChestnutHill,68.80.248.134,f8:8b:37:b1:91:a7,TG1682G,xfinitywifi_5GHz_HHS_BWG,103,9c:04:eb:65:83:b2,13580998,50645,...,40.074132,-75.200463,Freedom,Chestnut Hill,Secondary,NaN,home_hotspot,POINT (-75.200463 40.074132),f8:8b:37:b1:91:a8,America/New_York
3,2020-05-03 00:15:00,ChestnutHill,68.80.248.134,f8:8b:37:b1:91:a7,TG1682G,xfinitywifi_5GHz_HHS_BWG,103,9c:04:eb:65:83:b2,26734653,183857,...,40.074132,-75.200463,Freedom,Chestnut Hill,Secondary,NaN,home_hotspot,POINT (-75.200463 40.074132),f8:8b:37:b1:91:a8,America/New_York
4,2020-05-03 00:20:00,ChestnutHill,68.80.248.134,f8:8b:37:b1:91:a7,TG1682G,xfinitywifi_5GHz_HHS_BWG,103,9c:04:eb:65:83:b2,28419088,229363,...,40.074132,-75.200463,Freedom,Chestnut Hill,Secondary,NaN,home_hotspot,POINT (-75.200463 40.074132),f8:8b:37:b1:91:a8,America/New_York


In [39]:
# for item in result:
#     # create dataframe
#     df = pd.merge(s3_read_parquet(item), gis_df, on='ap_mac_address')
#     # save dataframe to parquet
#     df.to_parquet(pickle_path)

In [47]:
df.to_parquet('./file4.parquet')

In [49]:
import glob
parquet_files = [file for file in glob.glob('*.parquet')]

In [50]:
parquet_files

['file1.parquet', 'file2.parquet', 'file3.parquet', 'file4.parquet']

In [51]:
import shutil
import os

for file in parquet_files:
    shutil.move(file, pickle_path)